In [ ]:
%matplotlib notebook
import firedrake

### Advection

The previous examples showed *elliptic* problems like the Poisson equation, where the solution is very smooth, you can use about any continuous shape functions, and you get nice symmetric positive-definite matrices.
But the night is dark and full of terrors, or, hyperbolic conservation laws.
These are the kinds of problems that the finite volume method was invented for.
We'll start with the advection equation

$$\frac{\partial}{\partial t}h + \nabla\cdot hu = 0$$

for the field $h$, where $u$ is a velocity field.

The **discontinuous Galerkin** method is a way to solve problems like the advection equation by using discontinuous shape functions.
It's similar in many respects to the finite volume method and actually includes the FVM as a special case.
First, we'll create the velocity field describing solid-body rotation about the origin.

In [ ]:
mesh = firedrake.UnitDiskMesh(4)
x = firedrake.SpatialCoordinate(mesh)
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=1)
u = firedrake.interpolate(firedrake.as_vector((-x[1], x[0])), V)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
firedrake.streamplot(u, axes=axes)

To create a discontinuous function space, we pass the argument `'DG'` for discontinuous Galerkin as the `family`.
Here we'll use degree 0 polynomials, i.e. piecewise constant in each cell.

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='DG', degree=0)

Next we need to pick an initial condition.
For discontinuous spaces, we use the `project` function instead of `interpolate`.

In [ ]:
from firedrake import inner, sqrt, min_value, max_value
x0 = firedrake.Constant((1/2, 0.))
r = sqrt(inner(x - x0, x - x0))
R = firedrake.Constant(1/4)
h0 = firedrake.project(max_value(1 - r / R, 0), Q)

In [ ]:
fig, axes = plt.subplots()
triangles = firedrake.tripcolor(h0, axes=axes)
fig.colorbar(triangles)

The weak form of the advection equation is more involved than for, say, the Poisson equation.
It involves integration not just over the whole domain, but over all the edges $E$ of the computational mesh.
We also have to pick a numerical flux function $F$ which depends on the solution $h$, the velocity $u$, and the normal vector $n$ to each cell of the mesh.
**If we're using upwinding, then the numerical flux is**

$$F = h\max\{u\cdot n, 0\}.$$

This is by far the most common choice but there are others, e.g. approximate Riemann solvers.

Let $u_+$, $h_+$, $n_+$ be the values of the velocity, thickness, and unit normal vector on one side of the edge $E$, with a $-$ subscript representing the values on the other side.
Then the main part of the weak form is:

$$\int_\Omega\frac{\partial h}{\partial t}q\, dx + \sum_E\int_E\left\{F(h_+, u_+, n_+) - F(u_-, h_-, n_-)\right\}\cdot\{q_+ - q_-\}\,dS + \ldots = 0$$

for all test functions $q$.
**If we pick some cell $K$ of the mesh and take $q = 1$ in $K$ and $q = 0$ elsewhere, this discretization is the same as the finite volume method!**
The interesting things happen when we use higher degree polynomials, which I'll show later.

(There's a bit of a subtlety here in that you don't really know which is the + side and which is the - side, but the form you write down has to come out the same if you flip all the signs.
So if it matters you did something wrong!)

We left off some boundary terms in the last equation to focus on the fluxes, so let's figure out the rest now.
Let $h_\text{in}$ be the value of $h$ on the inflow boundary; the boundary forcing is:

$$\ldots = \int_{\partial\Omega} h_{\text{in}}\min\{u\cdot n, 0\}q\, ds + \int_{\partial\Omega} h\max\{u\cdot n, 0\}q\, ds.$$

We've now got all the pieces in place to actually express our problem.

##### Show me the code already

First, we'll import some more members from Firedrake.
In the last example, we used the measure `dx` to represent integration over the entire spatial domain.
Here we also need a measure `ds` to represent integration over the edges on the boundary of the domain, and a measure `dS` for integration over all of the interior edges.

In [ ]:
from firedrake import grad, dx, ds, dS

We've already seen several times how to get a symbolic object from a mesh that represents points inside the domain.
To solve hyperbolic problems, we'll also need a symbolic object to represent the unit outward normal vector to a cell or to the domain boundary:

In [ ]:
n = firedrake.FacetNormal(mesh)

Once we've created variables for the solution and for the test function we can create all the parts of the weak form in a way that looks nearly identical to the math.

In [ ]:
h = h0.copy(deepcopy=True)
q = firedrake.TestFunction(Q)

F = h * max_value(inner(u, n), 0)
edge_flux = (F('+') - F('-')) * (q('+') - q('-')) * dS

in_flux = h0 * min_value(inner(u, n), 0) * q * ds
out_flux = h * max_value(inner(u, n), 0) * q * ds

dh_dt = -(edge_flux + in_flux + out_flux)

p = firedrake.TrialFunction(Q)
M = p * q * dx

Now to actually solve it, we need a timestep $\delta t$ that satisfies the Courant-Friedrichs-Lewy condition:

$$\frac{\delta x}{\delta t} > \max|u|,$$

where $\delta x$ is the smallest diameter of any cell of the mesh.

In [ ]:
import numpy as np
dx_min = mesh.cell_sizes.dat.data_ro.min()
u_max = 1.
timestep = dx_min / u_max / 2

final_time = 2 * np.pi
num_steps = int(final_time / timestep) + 1
dt = firedrake.Constant(final_time / num_steps)

This is magic and you can ignore it.
If you want me to explain it I will.

In [ ]:
parameters = {
    'solver_parameters': {
        'ksp_type': 'preonly',
        'pc_type': 'bjacobi',
        'sub_pc_type': 'ilu'
    }
}

Rather than solve the PDE for $h$ directly, we'll solve for the increment over one timestep.
(When you use Runge-Kutta methods this makes things much easier.)

In [ ]:
import tqdm
dh = firedrake.Function(Q)
for step in tqdm.trange(num_steps):
    firedrake.solve(M == dt * dh_dt, dh, **parameters)
    h.assign(h + dh)

The result looks pretty diffused out, which is exactly what we expect for the first-order finite volume method.

In [ ]:
fig, axes = plt.subplots()
triangles = firedrake.tripcolor(h, axes=axes)
fig.colorbar(triangles)

To machine precision, the total mass is conserved though, so that's a good *smoke test*.

In [ ]:
firedrake.assemble((h - h0) * dx)

Let's see if we can get something more accurate.

### Higher order

We could get a more accurate simulation by increasing the number of mesh cells.
Alternatively, we could use higher-degree basis functions, which I'll show below.
First, we'll create a new function space and project the initial data into that space.

In [ ]:
Q1 = firedrake.FunctionSpace(mesh, family='DG', degree=1)
h1 = firedrake.project(max_value(1 - r / R, 0), Q1)

In [ ]:
fig, axes = plt.subplots()
triangles = firedrake.tripcolor(h1, axes=axes)
fig.colorbar(triangles)

Much of what we did before still applies.

In [ ]:
h = h1.copy(deepcopy=True)
q = firedrake.TestFunction(Q1)

F = h * max_value(inner(u, n), 0)
edge_flux = (F('+') - F('-')) * (q('+') - q('-')) * dS

in_flux = h1 * min_value(inner(u, n), 0) * q * ds
out_flux = h * max_value(inner(u, n), 0) * q * ds

But with higher-degree elements there's one extra bit!
In addition to the material flux across element boundaries, there's also flux within a cell itself.
The main part of the weak form for higher-degree elements is:

$$\int_\Omega\left(\frac{\partial h}{\partial t}q - \underbrace{hu\cdot\nabla q}_{\text{cell flux}}\right)dx + \sum_E\int_E(F_+ - F_-)(q_+ - q_-)dS + \ldots = 0$$

This weak form also works perfectly well when the shape functions are degree 0, but the gradient of a test function within each cell is zero, so the internal cell fluxes are trivial.

In [ ]:
cell_flux = -h * inner(u, grad(q)) * dx

There's one extra term in the rate of change of $h$.

In [ ]:
dh_dt = -(cell_flux + edge_flux + in_flux + out_flux)

p = firedrake.TrialFunction(Q1)
M = p * q * dx

We have to be more careful about the timestep -- the CFL condition is *much* more stringent for higher degree elements.
Here we're using a fudge factor of 16 rather than 2.

In [ ]:
timestep = dx_min / u_max / 16

final_time = 2 * np.pi
num_steps = int(final_time / timestep) + 1
dt = firedrake.Constant(final_time / num_steps)

Observe how each timestep is more expensive and we have to take more of them!

In [ ]:
dh = firedrake.Function(Q1)
for step in tqdm.trange(num_steps):
    firedrake.solve(M == dt * dh_dt, dh, **parameters)
    h.assign(h + dh)

But the results are a lot better.

In [ ]:
fig, axes = plt.subplots()
triangles = firedrake.tripcolor(h, axes=axes)
fig.colorbar(triangles)

The relative error in the 1-norm is much smaller.

In [ ]:
firedrake.assemble(abs(h - h1) * dx) / firedrake.assemble(h1 * dx)

But observe how the solution undershoots now -- it's taken on negative values, despite the fact that the initial value is strictly positive.
To remedy this, we need to look into limiters.